In [1]:
from ioh import get_problem, ProblemClass
from ioh import logger
import math
import os
from dataclasses import dataclass
import BA 
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
import matplotlib.pyplot as plt
import numpy as np
import torch
from sklearn.random_projection import GaussianRandomProjection
import botorch
import gpytorch
from gpytorch.constraints import Interval
from gpytorch.kernels import MaternKernel, ScaleKernel, RBFKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood
from torch.quasirandom import SobolEngine

from botorch.acquisition.analytic import LogExpectedImprovement
from botorch.exceptions import ModelFittingError
from botorch.fit import fit_gpytorch_mll
from botorch.generation import MaxPosteriorSampling
from botorch.models import SingleTaskGP
from botorch.optim import optimize_acqf
from sklearn.preprocessing import StandardScaler


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on {device}")
dtype = torch.float
SMOKE_TEST = os.environ.get("SMOKE_TEST")

# problem dimension
dim = 40 if not SMOKE_TEST else 50
projected_dim = 15

# initial sample points
n_init = 10 if not SMOKE_TEST else 4
max_cholesky_size = float("inf")

problem = get_problem(1, dimension=dim, instance=2, problem_class=ProblemClass.BBOB)

Running on cpu


In [3]:
EVALUATION_BUDGET = 100 if not SMOKE_TEST else 10
NUM_RESTARTS = 3 if not SMOKE_TEST else 2
RAW_SAMPLES = 51 if not SMOKE_TEST else 4
N_CANDIDATES = min(dim, max(200, 20 * dim)) if not SMOKE_TEST else 4

In [7]:
S = BA.multi_scale_embedding(6, 3)
X = torch.randint(100, (7, 2))
print("0")
print(f"S before increase\n{S}")
print(f"X before increase\n{X}")

0
S before increase
tensor([[ 0.9301,  0.0091,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.8765, -0.3164,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.4450,  0.9531]])
X before increase
tensor([[83,  0],
        [68, 48],
        [81, 30],
        [65, 91],
        [94, 26],
        [97, 25],
        [71, 82]])


In [17]:
state = BA.BaxusState(dim=dim, eval_budget=EVALUATION_BUDGET - n_init)
n_sample = 40
X_sample = BA.get_initial_points(state.d_init, n_sample)
S = BA.multi_scale_embedding(input_dim=state.dim, target_dim=state.d_init)
print(state.dim,  state.d_init)
print(S.shape)
X_baxus_target = BA.get_initial_points(state.d_init, n_init)
X_baxus_input = X_baxus_target @ S
Y_baxus = torch.tensor(
        [-problem(list(x)) for x in X_baxus_input], dtype=dtype, device=device
    ).unsqueeze(-1)

with botorch.settings.validate_input_scaling(False):
    for _ in range(EVALUATION_BUDGET - n_init):  # Run until evaluation budget depleted
            # Fit a GP model
            train_Y = (Y_baxus - Y_baxus.mean()) / Y_baxus.std()
            likelihood = GaussianLikelihood(noise_constraint=Interval(1e-8, 1e-3))
            model = SingleTaskGP(
                X_baxus_target, train_Y, likelihood=likelihood
            )
            mll = ExactMarginalLogLikelihood(model.likelihood, model)

            # Fit the model using Cholesky context
            with gpytorch.settings.max_cholesky_size(max_cholesky_size):
                try:
                    fit_gpytorch_mll(mll)
                except ModelFittingError:
                    # Use Adam-based optimization if Cholesky decomposition fails
                    optimizer = torch.optim.Adam([{"params": model.parameters()}], lr=0.1)
                    for _ in range(200):
                        optimizer.zero_grad()
                        output = model(X_baxus_target)
                        loss = -mll(output, train_Y.flatten())
                        loss.backward()
                        optimizer.step()

                # Generate new candidates
                X_next_target = BA.create_candidate(
                    state=state,
                    model=model,
                    X=X_baxus_target,
                    Y=train_Y,
                    device=device,
                    dtype=dtype,
                    n_candidates=N_CANDIDATES,
                    num_restarts=NUM_RESTARTS,
                    raw_samples=RAW_SAMPLES,
                    acqf="ts",
                )
            
            # Map new candidates to high-dimensional space
            X_next_input = X_next_target @ S
            Y_next = torch.tensor(
                [-problem(list(x)) for x in X_next_input], dtype=dtype, device=device
            ).unsqueeze(-1)

            # Update state and concatenate new points
            state = BA.update_state(state=state, Y_next=Y_next)
            X_baxus_input = torch.cat((X_baxus_input, X_next_input), dim=0)
            X_baxus_target = torch.cat((X_baxus_target, X_next_target), dim=0)
            Y_baxus = torch.cat((Y_baxus, Y_next), dim=0)
            
            print(
                f"iteration {len(X_baxus_input)}, d={len(X_baxus_target.T)})  Best value: {state.best_value:.3}, TR length: {state.length:.3}"
            )

            if state.restart_triggered:
                state.restart_triggered = False
                print("increasing target space")

                S, X_baxus_target = BA.increase_embedding_and_observations(
                    S, X_baxus_target, state.new_bins_on_split
                )
                print(f"new dimensionality: {len(S)}")
                
                state.target_dim = len(S)
                state.length = state.length_init
                state.failure_counter = 0
                state.success_counter = 0


40 1
torch.Size([1, 40])
iteration 11, d=1)  Best value: -6.59e+02, TR length: 0.4
iteration 12, d=1)  Best value: -6.59e+02, TR length: 0.2
iteration 13, d=1)  Best value: -6.59e+02, TR length: 0.1


/var/folders/yk/hp80cpj103xfzhptqczygcv40000gn/T/ipykernel_16829/4007967345.py:18: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  model = SingleTaskGP(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/var/folders/yk/hp80cpj103xfzhptqczygcv40000gn/T/ipykernel_

iteration 14, d=1)  Best value: -6.59e+02, TR length: 0.05
iteration 15, d=1)  Best value: -6.59e+02, TR length: 0.025
iteration 16, d=1)  Best value: -6.59e+02, TR length: 0.0125
iteration 17, d=1)  Best value: -6.59e+02, TR length: 0.00625
increasing target space
new dimensionality: 2
iteration 18, d=2)  Best value: -6.59e+02, TR length: 0.4
iteration 19, d=2)  Best value: -6.59e+02, TR length: 0.2


/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/var/folders/yk/hp80cpj103xfzhptqczygcv40000gn/T/ipykernel_16829/4007967345.py:18: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  model = SingleTaskGP(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-pac

iteration 20, d=2)  Best value: -6.57e+02, TR length: 0.2
iteration 21, d=2)  Best value: -6.57e+02, TR length: 0.1
iteration 22, d=2)  Best value: -6.57e+02, TR length: 0.05
iteration 23, d=2)  Best value: -6.57e+02, TR length: 0.025
iteration 24, d=2)  Best value: -6.57e+02, TR length: 0.0125


...

/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-04 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10

iteration 94, d=16)  Best value: -5.64e+02, TR length: 0.4
iteration 95, d=16)  Best value: -5.64e+02, TR length: 0.4
iteration 96, d=16)  Best value: -5.62e+02, TR length: 0.4
iteration 97, d=16)  Best value: -5.59e+02, TR length: 0.4


/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-04 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-03 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-02 to the diagonal
  warnings.warn(
/var/folders/yk/hp80cpj103xfzhptqczygcv40000gn/T/ipykernel_16829/4007967345.py:18: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision 

iteration 98, d=16)  Best value: -5.58e+02, TR length: 0.4
iteration 99, d=16)  Best value: -5.58e+02, TR length: 0.4
iteration 100, d=16)  Best value: -5.58e+02, TR length: 0.4


/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-04 to the diagonal
  warnings.warn(
/Users/jiaxuanyu/miniconda3/envs/BO/lib/python3.10